In [ ]:
import requests
import re
import json
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
def extract_card_info(card_id):
    url = f"https://api.card-gorilla.com:8080/v1/cards/{card_id}"
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)

    if res.status_code != 200:
        return None

    data = res.json()

    # 카드 기본 정보
    card_idx = data.get("idx")
    card_name = data.get("name")

    # 신규 발급 중단 여부
    is_discon = data.get("is_discon", False)
    if is_discon:
        return None

    card_company = data.get("corp", {}).get("name")

    # 카드 브랜드
    brands = data.get("brand", [])
    card_brand = brands[0]["name"] if brands else "없음"

    # 국내/해외 연회비 추출
    annual_fee_detail = data.get("annual_fee_detail") or ""
    domestic_fee = re.search(r'국내.*?([0-9,]+)원', annual_fee_detail)
    domestic_fee = int(domestic_fee.group(1).replace(",", "")) if domestic_fee else 0
    foreign_fee = re.search(r'해외.*?([0-9,]+)원', annual_fee_detail)
    foreign_fee = int(foreign_fee.group(1).replace(",", "")) if foreign_fee else 0

    # 카드 혜택
    benefits_dict = {}
    key_benefit = data.get("key_benefit", [])
    for benefit in key_benefit:
        title = benefit.get("title")
        comment = benefit.get("comment") or ""
        info = benefit.get("info") or ""
        
        # 주의사항인 경우: 전체 내용 추출
        if "유의사항" in title:
            if info:
                soup = BeautifulSoup(info, "html.parser")
                desc = soup.get_text().strip().replace('\n', ' ').replace('\xa0', ' ')  # 줄바꿈을 공백으로 대체
            elif comment:
                desc = comment
            else:
                desc = "내용 없음"
                
        # 일반 혜택 : 내용 1줄 or 100자 under
        else:
            if comment:
                desc = comment
            elif info:
                soup = BeautifulSoup(info, "html.parser")
                text = soup.get_text().strip()
                desc = text.split("\n")[0].strip() if text else ""
                if len(desc) > 100:
                    desc = desc[:100] + "..."
            else:
                desc = "설명 없음"
            
        benefits_dict[title] = desc

    return {
        "카드_url": "https://www.card-gorilla.com/card/detail/" + str(card_idx),
        "카드이름": card_name,
        "카드사": card_company,
        "카드브랜드": card_brand,
        "국내연회비": domestic_fee,
        "해외연회비": foreign_fee,
        "주요혜택": benefits_dict
    }

# 수집할 카드 ID 범위 지정 (예: 1~2857)
card_infos2 = []
for card_id in tqdm(range(1,2858)):
    info = extract_card_info(card_id)
    if info:
        card_infos2.append(info)

In [ ]:
card_infos2

[{'카드_url': 'https://www.card-gorilla.com/card/detail/8',
  '카드이름': '신한카드 The CLASSIC-Y',
  '카드사': '신한카드',
  '카드브랜드': 'VISA',
  '국내연회비': 0,
  '해외연회비': 100000,
  '주요혜택': {'선택형': 'Gift Option Service - 매년 1회 옵션 품목 중 한 가지를 선택하여 이용',
   '모든가맹점': '일시불/할부 이용금액의 0.7% 마이신한포인트 적립\n해외, 명품 아울렛(신세계/롯데), 면세점, 골프 업종 이용 시 5% 마이신한포인트 적립',
   '주유소': 'GS칼텍스 주유 시 리터당 60원 결제일 할인 ',
   '생활': '커피/제과/택시/화장품/영화 5% 할인 ',
   '면세점': '제주 JDC 면세점 이용 시 8% 할인',
   '진에어': '진에어 국제선 항공권 결제일 할인',
   '공연/전시': '오디오 가이드 및 오페라 글라스 무료 이용 서비스',
   '프리미엄': '국제브랜드사 서비스 공통사항 안내',
   '유의사항': '- 계약 체결 전 상품에 관한 사항은 상품설명서 및 약관을 읽어보시기 바랍니다.※ 카드이용시 제공되는 포인트 및 할인혜택 등의 부가서비스는 카드 신규출시(2014.10.31) 이후 3년 이상 축소, 폐지 없이 유지됩니다.※ 상기에도 불구하고, 다음과 같은 사유가 발생한 경우 카드사는 부가서비스를 변경할 수 있습니다.     ① 카드사의 휴업·파산·경영상의 위기 등에 따른 불가피한 경우     ② 제휴업체의 휴업•파산•경영상의 위기로 인해 불가피하게 부가서비스를 축소•변경하는 경우로서 다른 제휴업체를 통해 동종의 유사한 부가서비스 제공이 불가한 경우     ③ 제휴업체가 카드사의 의사에 반하여 해당 부가서비스를 축소하거나 변경 시, 당초 부가서비스에 상응하는 다른 부가서비스를 제공하는 경우     ④ 부가서비스를 3년 이상 제공한 상태에서 해당 부가서비스로 인해 카드의 수익성이 현저히 낮

In [ ]:
# json save
with open("card_infos_short.json", "w", encoding="utf-8") as f:
    json.dump(card_infos2, f, ensure_ascii=False, indent=4)